In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib qt 

import tifffile 
import napari
import os
import numpy as np
import regions
import matplotlib.pyplot as plt

from utils import grouped_z_project

In [6]:


# stack1 = tifffile.imread(
#     "/Users/jamesrowland/Desktop/J007_2x_rightHem_140deep_00001.tif"
# )
from utils import load_tiff_folder, load_tiff_folder_truncated


stack = load_tiff_folder_truncated("/Volumes/MarcBusche/Josef/2P/to_analyse/2024-10-22/JB011/suite2p/plane0/reg_tif", 20)
grouped = grouped_z_project(stack, 10)
average = np.mean(stack, axis=0)


In [7]:
viewer = napari.Viewer()
viewer.add_image(average, name="Average Image")

viewer.add_image(grouped, name='Grouped z project')
viewer.add_image(stack, name='raw')

<Image layer 'raw' at 0x1488b8860>

In [27]:
if point_layer := [layer for layer in viewer.layers if layer.name == "Points"]:
    point_layer = point_layer[0]
else:
    point_layer = viewer.add_points(name="Points")

In [28]:
rois = point_layer.data[0]
assert rois.size > 0, "You need to add a point in the viewer"

In [29]:
roi0_center = regions.PixCoord(rois[2], rois[1])
roi0_circle = regions.CirclePixelRegion(center=roi0_center, radius=10) #radius is in px
roi0_patch = roi0_circle.as_artist(facecolor='none', edgecolor='yellow', linewidth=2)
roi0_mask = roi0_circle.to_mask().to_image(stack[0, :, :].shape)

fig, ax = plt.subplots(1, 2, figsize=(10,5))

im0 = ax[0].imshow(average)
ax[0].add_patch(roi0_patch)

im1 = ax[1].imshow(roi0_mask, cmap='Greys_r')

In [30]:
mask_activity = stack * roi0_mask
mask_activity[mask_activity == 0] = np.nan
mean_mask = np.nanmean(mask_activity, axis=(1,2))

In [31]:

plt.figure()
sampling_rate = 30
x_axis = np.linspace(0, mask_activity.shape[0]/sampling_rate, mask_activity.shape[0])
# x_axis = np.arange(mask_activity.shape[0])
plt.plot(x_axis, np.nanmean(mask_activity, axis=(1,2)))
plt.xlabel("Time (s)")

Text(0.5, 0, 'Time (s)')